# L3b: Praktyczny Machine Learning w Pythonie
<br>
<img src="figures/L1/dilbert-2213.gif">

## Rozpoznawanie cyfr

Nasz model będzie składał się z 2 kroków:
1. Zmniejszenie ilości wymiarów
2. Klasyfikacja modelem liniowym

Najpierw jednak musimy zapoznać się ze zbiorem danych.

In [1]:
from sklearn.datasets import fetch_mldata
mnist = fetch_mldata('MNIST original')

URLError: <urlopen error [Errno 60] Operation timed out>

### MNIST

<img src="figures/L1/mnist_originals.png">

MNIST to baza danych odręcznie napisanych cyfr około 20 lat temu. Ludzie są w stanie rozpoznać ok. 99,5% cyfr z tego zbioru poprawnie.
Zobaczymy ile nam się uda!

In [4]:
img = mnist.data[0]
print "Pierwsz obrazek z ", mnist.data.shape[0], ":", img # Pixele

NameError: name 'mnist' is not defined

In [ ]:
# Możemy sobie narysować wcześniej wypisaną cyfrę
import matplotlib.pylab as plt
%matplotlib inline
plt.imshow(img.reshape(28,28), cmap="gray")

In [ ]:
import sklearn
from sklearn import linear_model, decomposition
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
import numpy as np

### Krok 1: wczytanie i podzielenie danych

Skalowanie jest bardzo ważne. Dzięki temu mamy średnią danej cechy 0 i wariancję 1 (mówiąc po ludzku, dane będą bardziej  przypominały kulke w przestrzeni wejściowej)

<img src="figures/L1/prepro2.jpeg" width=600>

In [ ]:
#Wczytujemy dane i skalujemy
X, Y = mnist.data.astype("float64"), mnist.target 
X = preprocessing.scale(X)

Tak jak wspomniałem algorytm trenujemy na innych danych niż testujemy. **To bardzo ważne**. Do każdych danych da się dopasować taki model, który idealnie na nich odpowiada (np. poprzez zapamiętanie wszystkich przykładów). Jeśli tak zrobimy, to mówimy że nasz model **zoverfitował**.

<img src="figures/L1/underfitting-overfitting.png">

In [ ]:
#Dzielimy na dane trenujące i testujące
X_train, X_test, Y_train, Y_test = train_test_split(X,Y)

### Krok 2: dopasujmy pare modeli

In [ ]:
# Na wszystkich przykladach osiaga 92% dokladnosci. 
N = 500 # Podzbiór danych
model = LogisticRegression(C=0.001) # Model z domyślnymi parametrami
model.fit(X_train[0:N], Y_train[0:N])

Y_test_predicted = model.predict(X_test)
print "Dokładność modelu wytrenowanego na ",N, " przykladach to: ",100*sklearn.metrics.accuracy_score(Y_test, Y_test_predicted), "%"

In [ ]:
# Na wszystkich przykladach osiaga 92% dokladnosci. 
N = 500 # Podzbiór danych
model = LogisticRegression(C=1) # Mniej regularyzacji, a taka sama dokladnosc
model.fit(X_train[0:N], Y_train[0:N])

Y_test_predicted = model.predict(X_test)
print "Dokładność modelu wytrenowanego na ",N, " przykladach to: ",100*sklearn.metrics.accuracy_score(Y_test, Y_test_predicted), "%"

In [ ]:
print "Przyklad zaklasyfikowany jako ", model.predict(X_test[5])
plt.imshow(X_test[5].reshape(28,28), cmap="gray")

### Cwiczenie 1 (1 pkt)

Prosze narysowac wykres (plt.plot) dokladnosci w zaleznosci od wartosci C na zbiorze treningowym i testowym (dla tych N=500 przykladow)

In [ ]:
#niestety baza danych się nie pobiera, więc nie ma możliwości sprawdzenia poprawności cw1 oraz cw2
N = 500 
M = 100
C_array = np.linspace(0.001, 10, M)
accuracy_array = np.empty(M)
for i, C in enumarate(C_array):
    model = LogisticRegression(C) 
    model.fit(X_train[0:N], Y_train[0:N])

    Y_test_predicted = model.predict(X_test)
    accuracy_array[i] = 100*sklearn.metrics.accuracy_score(Y_test, Y_test_predicted)

plt.plot(C_array, accuracy_array)

### Cwiczenie 2 (2 pkt)

(Punkt za narysowanie oraz punkt za poprawne wytłumaczenie)

Proszę narysować jak wyglądaja nauczone wagi modelu dla różnych wartości parametru C (należy użyc plt.imshow). Z czego wynikają różnice w narysowanych obrazkach? 

Uwaga: wagi modelu regresji logistycznej są wymiaru (10, 28*28). Aby narysować wagi odpowiadające klasie "3" należy pobrać w[4]. Aby narysować to z użyciem imshow należy zrobić w[4].reshape(28, 28)

In [ ]:
m = 10
c_array = np.linspace(0.001, 10, M)
for c in c_array:
    model = LogisticRegression(C) 
    model.fit(X_train[0:N], Y_train[0:N])
    
    plt.imshow(model.coef_, cmap="gray")

### Krok 3: Tworzymy caly model zmniejszając wielkość przykładów

W scikit-learn możemy połączyć pare modeli w **pipeline**, który sam implementuje interfejs **Estimator**.

In [ ]:
pca = decomposition.PCA(n_components=20)
pca.fit(X_train)
X_train_transf = pca.transform(X_train)

In [ ]:
logistic = linear_model.LogisticRegression(C=0.1)
logistic.fit(X_train_transf, Y_train)

In [ ]:
pipe = Pipeline(steps=[('pca', pca), ('logistic', logistic)])

In [ ]:
# Zobaczmy na dokładność modelu
Y_test_predicted = pipe.predict(X_test)
print "Dokładność modelu wytrenowanego to: ",100*sklearn.metrics.accuracy_score(Y_test, Y_test_predicted), "%"

# Deep Learning

Jak widać, na rozważanych zbiorach, można osiągnąć wysoką dokładność (>85%) używając prostych modeli. Niestety na bardziej skomplikowanych danych (np. CIFAR-100, https://www.cs.toronto.edu/~kriz/cifar.html) nie jest już tak prosto.

Tradycyjne architektury, które były popularne do niedawna, używały ręcznej ekstrakcji "lepszych" cech:

<img src="figures/L1/standard.jpg">

Deep Learning jest poddziedziną machine learningu, gdzie rozważamy architektury, które samoistnie uczą się cech na tyle dobrych, że dokładnie takie same modele jak Państwu pokazywaliśmy działają z zadowalającą precyzją

<img src="figures/L1/features.png">